In [24]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
============================================================
CHOROPLETH CÔTE D'IVOIRE
============================================================

Description:
    Génère une carte choroplèthe (PNG 600 DPI, SVG vectoriel,
    HTML interactif) à partir d'un fichier CSV.

Auteur:
    Dramane Dagnogo

Organisation:
    Data Ivoire

Email:
    dagnogo.data.ivoire@gmail.com

Date de création:
    2026-02-15

Version:
    1.0.0

Licence:
    libre

Dépendances:
    pandas
    geopandas
    matplotlib
    folium
    requests

Usage:
    plot_choropleth("data.csv", element="region")

============================================================
"""

import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import requests
import zipfile
import io
import folium
import unicodedata


# ---------------- NORMALISATION DES NOMS DES INDEX----------------
def normalize(s):
    s = unicodedata.normalize('NFKD', str(s))
    s = ''.join(c for c in s if not unicodedata.combining(c))
    s = s.upper()
    s = s.replace('-', ' ')
    s = ' '.join(s.split())
    return s


# ---------------- TELECHARGEMENT DU FICHIER SHAPEFILE (CARTOGRAPHIE) ----------------
def download_shapefile_from_github(raw_url, extract_to="./shapefiles"):
    os.makedirs(extract_to, exist_ok=True)
    r = requests.get(raw_url)
    if r.status_code != 200:
        raise ValueError(f"Impossible de télécharger le fichier : {raw_url}")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(extract_to)
    return extract_to


# ---------------- FONCTION PRINCIPALE ----------------
def plot_choropleth1(
    data_path,
    element='region',
    data_column=None,
    cmap='Purples',
    figsize=(45, 22),
    output_basename="choropleth_output",
    sep=";",
    column_name="REGION",
    surfixe_montant=''
):

    # ---------- CHARGEMENT DES DONNEES  ----------
    data = pd.read_csv(data_path, sep=sep)

    base_url = "https://github.com/dagnogodataivoire-prog/choroplete_ivoire/raw/main/"

    shapefiles = {
        "CIV": "gadm41_CIV_0.zip",
        "district": "gadm41_CIV_1.zip",
        "region": "gadm41_CIV_2.zip",
        "departement": "gadm41_CIV_3.zip",
        "sous_prefecture": "gadm41_CIV_4.zip"
    }

    label_cols = {
        "CIV": "NAME_0",
        "district": "NAME_1",
        "region": "NAME_2",
        "departement": "NAME_3",
        "sous_prefecture": "NAME_4"
    }

    if element not in shapefiles:
        raise ValueError("Element inconnu.")

    # ---------- LECTURE DU SHAPEFILE ----------
    shapefile_dir = download_shapefile_from_github(base_url + shapefiles[element])
    shp_files = [f for f in os.listdir(shapefile_dir) if f.endswith(".shp")]
    shp_path = os.path.join(shapefile_dir, shp_files[0])
    regions = gpd.read_file(shp_path)

    label_col = label_cols[element]

    # ---------- NORMALISATION DES DEUX FFICHIERS  ----------
    data['REGION_clean'] = data[column_name].apply(normalize)
    regions['NAME_clean'] = regions[label_col].apply(normalize)

    # ---------- AUTODETECTION DE LA COLONE A VISUALISER ----------
    if data_column is None:
        print("Tu n'as pas definit la valeur de la colonne a visualiser dans : 'data_colum'")

        data_column = data.columns[-1]

    data[data_column] = (
    data[data_column]
    .astype(str)
    .str.replace(" ", "")
    .str.replace(",", ".", regex=False))

    data[data_column] = pd.to_numeric(data[data_column], errors="coerce")

    # ---------- MERGING ----------
    regions1 = regions.merge(
        data,
        left_on='NAME_clean',
        right_on='REGION_clean',
        how='left'
    )

    # ---------------- CREATION DE LA FIGURE AVEC MATPLOTLIB ----------------
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    regions1.plot(
        column=data_column,
        cmap=cmap,
        legend=False,
        ax=ax,
        edgecolor="white",
        linewidth=0.9
    )

    for _, row in regions1.iterrows():
        if pd.notna(row[data_column]):
            x, y = row.geometry.representative_point().coords[0]
            ax.text(
                x,
                y,
                f"{row[label_col]}\n{row[data_column]}{surfixe_montant}",
                ha="center",
                va="center",
                fontsize=9,
                weight="bold"
            )

    vmin = regions1[data_column].min()
    vmax = regions1[data_column].max()

    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("top", size="3%", pad=0.1)

    cbar = fig.colorbar(sm, cax=cax, orientation="horizontal")
    cbar.set_ticks([vmin, vmax])
    cbar.set_ticklabels([f"{vmin:.1f}", f"{vmax:.1f}"])

    ax.axis("off")

    fig.savefig(f"{output_basename}.png", dpi=600, bbox_inches='tight')
    fig.savefig(f"{output_basename}.svg", dpi=600, bbox_inches='tight')
    plt.close(fig)

    print("PNG et SVG sauvegardés.")

    # ---------------- HTML AVEC FOLIUM ----------------
    regions1 = regions1.to_crs(epsg=4326)

    m = folium.Map(location=[6.5, -5.5], zoom_start=6, tiles="cartodbpositron")

    folium.Choropleth(
        geo_data=regions1,
        data=regions1,
        columns=[label_col, data_column],
        key_on=f"feature.properties.{label_col}",
        fill_color=cmap,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=data_column
    ).add_to(m)

    m.save(f"{output_basename}.html")

    print("HTML sauvegardé.")


# ---------------- MESSAGE DE CHARGEMENT ----------------
if __name__ == "__main__":
    print("Script chargé. Utilise :")
    print("plot_choropleth('data.csv', element='region')")

Script chargé. Utilise :
plot_choropleth('data.csv', element='region')


In [25]:
plot_choropleth1(data_path="/content/Population électorale 2025.csv",data_column="POPULATION ELECTORALE",
                 output_basename="civ_choropleth2_pop", cmap="Blues"
)

PNG et SVG sauvegardés.
HTML sauvegardé.


In [22]:
help(plot_choropleth)
#

Help on function plot_choropleth in module __main__:

plot_choropleth(data_path, element='region', data_column=None, merge_kwargs={'left_index': True, 'right_index': True}, cmap='Purples', figsize=(35, 12), output_basename='choropleth_output', sep=';', colun_name='REGION')

